In [398]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from scipy.stats import mode
from scipy.stats.distributions import randint, uniform, expon
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [62]:
train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')

In [210]:
X_train = train.drop('Survived', axis=1)
y_train = train['Survived'].ravel()

In [64]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [434]:
def write_test(df, mod, n=1):
    dft = pd.DataFrame(np.c_[df['PassengerId'],mod.predict(df)], columns=['PassengerId', 'Survived'])
    dft.to_csv('Predictions_Teste{}.csv'.format(n), index=False)

In [340]:
class Impute_Better(BaseEstimator, TransformerMixin):
    """ Impute missing data in a more effective way, groupwise by Sex and Pclass"""
    def __init__(self, col='Age', func=np.mean, mode=2):
        self.col = col
        self.func = func
        self.mode = mode
    def fit(self, X, y=None):
        if self.mode == 2:
            self.vals = X.groupby(by=['Sex', 'Pclass'])[self.col].agg(self.func)
        elif self.mode == 1:
            self.vals = X.groupby(by=['Sex'])[self.col].agg(self.func)
        else:
            self.vals = X[self.col].agg(self.func)
        return self
    def transform(self, X, y=None):
        Xt = X.copy()
        idx_na = X.loc[:,self.col].isna()
        if self.mode == 2:
            for i in range(len(self.vals)):
                Xt.loc[((X['Sex'] == self.vals.index[i][0]) & (X['Pclass'] == self.vals.index[i][1]) &
                       idx_na),self.col] = self.vals[i]
        elif self.mode == 1:
            for i in range(len(self.vals)):
                Xt.loc[((X['Sex'] == self.vals.index[i]) & idx_na),self.col] = self.vals[i]
        else:
            Xt.loc[idx_na, self.col] = self.vals
        return Xt

In [341]:
class New_Features(BaseEstimator, TransformerMixin):
    """ Add new features, in this case Parch + SibSp """
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        Xt = X.copy()
        Xt['Nrelatives'] = Xt['Parch'] + Xt['SibSp']
        return Xt

In [419]:
class Transf_Conti(BaseEstimator, TransformerMixin):
    """ Transforms a feature in categorical or startardizes it"""
    def __init__(self, n=4, q=False, conti=False):
        self.n = n
        self.q = q
        self.conti=conti
    def fit(self, X, y=None):
        if not self.conti:
            if not self.q:
                self.binst = pd.cut(X.values[:,0], bins=self.n, retbins=True, duplicates='drop')[1]
            if self.q:
                self.binst = pd.qcut(X.values[:,0], q=self.n, retbins=True, duplicates='drop')[1]
            self.higher = self.binst[-1]
            self.lower = self.binst[0]
        else:
            self.scaler_transf = StandardScaler()
            self.scaler_transf.fit(X)
        
        return self
    def transform(self, X, y=None):
        if not self.conti:
            val_ret = pd.cut(X.values[:,0], bins=self.binst, labels=False, include_lowest=True, duplicates='drop')
            val_ret[X.values[:,0] > self.higher] = self.n - 1
            val_ret[X.values[:,0] < self.lower] = 0
            return val_ret.reshape((-1, 1))
        else:
            return self.scaler_transf.transform(X)

In [420]:
class Select_Features(BaseEstimator, TransformerMixin):
    def __init__(self, select=False):
        self.select=select
    def fit(self, X, y=None):
        return self
    def transform(self, X,y=None):
        if not self.select:
            return X
        return X[:, self.select]

In [421]:
Pipe_Impute = Pipeline([
    #Impute Pipeline, the one which imputes missing data and creates new features
    ('Age_Impute', Impute_Better(col='Age', func=np.median)),
    ('Fare_Impute', Impute_Better(col='Fare', func=np.median)),
    ('Embarked_Impute', Impute_Better(col='Embarked', func=lambda x: mode(x)[0][0])),
    ('New_Features', New_Features())
])

In [422]:
Pipe_Preprocess = ColumnTransformer([
    ### Preprocessing pipeline. Encodes, standardizes
    ('Ordinal_Encode', OrdinalEncoder(), ['Sex','Pclass']), 
    ('Age_Encode', Transf_Conti(), ['Age']), 
    ('Fare_Encode', Transf_Conti(), ['Fare']), 
    ('Keep', 'passthrough', ['Parch', 'SibSp', 'Nrelatives']), 
    ('Embarked_Encode', OneHotEncoder(), ['Embarked']),
])

# 1 - Logistic Regression

In [432]:
FullPipePredict_LogRegression = Pipeline([
    ('Impute_Pipeline', Pipe_Impute),
    ('Preprocess_Pipeline', Pipe_Preprocess),
    ('Feature_Selection', Select_Features()),
    ('Predict', LogisticRegression(max_iter=1000, n_jobs=-1))
    #('Predict', RandomForestClassifier())
])

In [433]:
param_LogReg = {
    'Preprocess_Pipeline__Age_Encode__n':randint(3,7),
    'Preprocess_Pipeline__Age_Encode__q':randint(0,2),
    'Preprocess_Pipeline__Age_Encode__conti':randint(0,2),
    'Preprocess_Pipeline__Fare_Encode__n':randint(4,7),
    'Preprocess_Pipeline__Fare_Encode__conti':randint(0,2),
    'Preprocess_Pipeline__Fare_Encode__q':randint(0,2),
    'Feature_Selection__select':[[0,1,2,3,6]],
    'Predict__C':uniform(0.1, 2.5)
}

In [437]:
grid_LogReg = RandomizedSearchCV(FullPipePredict_LogRegression, param_distributions=param_LogReg, cv=5, n_iter=300, 
                                scoring='accuracy')

In [438]:
grid_LogReg.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('Impute_Pipeline',
                                              Pipeline(steps=[('Age_Impute',
                                                               Impute_Better(func=<function median at 0x00000298ABC0F550>)),
                                                              ('Fare_Impute',
                                                               Impute_Better(col='Fare',
                                                                             func=<function median at 0x00000298ABC0F550>)),
                                                              ('Embarked_Impute',
                                                               Impute_Better(col='Embarked',
                                                                             func=<function <lambda> at 0x00000298B6E07B80>)),
                                                              ('New_Features'...
                                 

In [444]:
grid_LogReg.best_score_

0.8069549934090766

In [442]:
best_logreg = grid_LogReg.best_estimator_

In [443]:
write_test(test, best_logreg, '_Best_LogReg2')

# 2 - SVC

In [459]:
FullPipePredict_SVC = Pipeline([
    ('Impute_Pipeline', Pipe_Impute),
    ('Preprocess_Pipeline', Pipe_Preprocess),
    ('Feature_Selection', Select_Features()),
    ('Predict', SVC(max_iter=1000000))
])

In [511]:
param_SVC = [{
    'Preprocess_Pipeline__Age_Encode__n':[6],
    'Preprocess_Pipeline__Fare_Encode__conti':[1],
    'Feature_Selection__select':[[0,1,2,3,6]],
    'Predict__C':uniform(0.2, 1.5),
    'Predict__kernel':['poly'],
    'Predict__degree':[3],
} 
]

grid_SVC = RandomizedSearchCV(FullPipePredict_SVC, param_distributions=param_SVC, cv=10, n_iter=200, 
                                scoring='accuracy')

In [512]:
grid_SVC.fit(X_train, y_train)

RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('Impute_Pipeline',
                                              Pipeline(steps=[('Age_Impute',
                                                               Impute_Better(func=<function median at 0x00000298ABC0F550>)),
                                                              ('Fare_Impute',
                                                               Impute_Better(col='Fare',
                                                                             func=<function median at 0x00000298ABC0F550>)),
                                                              ('Embarked_Impute',
                                                               Impute_Better(col='Embarked',
                                                                             func=<function <lambda> at 0x00000298B6E07B80>)),
                                                              ('New_Features...
                                 

In [516]:
grid_SVC.best_params_

{'Feature_Selection__select': [0, 1, 2, 3, 6],
 'Predict__C': 1.4610342371195217,
 'Predict__degree': 3,
 'Predict__kernel': 'poly',
 'Preprocess_Pipeline__Age_Encode__n': 6,
 'Preprocess_Pipeline__Fare_Encode__conti': 1}

In [517]:
SVC_best = grid_SVC.best_estimator_

In [518]:
write_test(test, SVC_best, 'SVC_PolyKernel3CV10')

# 2 - Random Forest

In [522]:
FullPipePredict_RF = Pipeline([
    ('Impute_Pipeline', Pipe_Impute),
    ('Preprocess_Pipeline', Pipe_Preprocess),
    ('Feature_Selection', Select_Features()),
    ('Predict', RandomForestClassifier(n_jobs=-1))
])

In [539]:
param_RF = {
    'Preprocess_Pipeline__Age_Encode__conti':[1],
    'Preprocess_Pipeline__Fare_Encode__conti':[1],
    'Feature_Selection__select':[[0,1,2,3,6]],
    'Predict__n_estimators':randint(100, 180),
    'Predict__max_leaf_nodes':randint(12,25)
}

In [540]:
grid_RF = RandomizedSearchCV(FullPipePredict_RF, param_distributions=param_RF, cv=10, n_iter=60,
                            scoring='accuracy')

In [541]:
grid_RF.fit(X_train, y_train)

RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('Impute_Pipeline',
                                              Pipeline(steps=[('Age_Impute',
                                                               Impute_Better(func=<function median at 0x00000298ABC0F550>)),
                                                              ('Fare_Impute',
                                                               Impute_Better(col='Fare',
                                                                             func=<function median at 0x00000298ABC0F550>)),
                                                              ('Embarked_Impute',
                                                               Impute_Better(col='Embarked',
                                                                             func=<function <lambda> at 0x00000298B6E07B80>)),
                                                              ('New_Features...
                   param_distribu

In [545]:
grid_RF.best_score_

0.8485642946317103

In [549]:
best_RF = grid_RF.best_estimator_

In [555]:
best_RF[-1].feature_importances_

array([0.45374799, 0.13817604, 0.14346081, 0.18145855, 0.08315661])

In [547]:
write_test(test, best_RF, '_RF_Well_Tuned')

In [556]:
joblib.dump(best_RF[-1], 'BestRFsofar.pkl')

['BestRFsofar.pkl']